In [1]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.9 MB/s eta 0:00:00


# Полная версия

In [2]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torch.utils.data import Dataset, DataLoader
from jiwer import cer

from tqdm import tqdm
import logging


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s: %(message)s',
    datefmt='%H:%M:%S'
)

DATA_DIR = '/kaggle/input/asr-numbers-recognition-in-russian' 
df_train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
df_train['transcription'] = df_train['transcription'].astype(str)
df_dev   = pd.read_csv(os.path.join(DATA_DIR, 'dev.csv'))
df_dev ['transcription'] = df_dev ['transcription'].astype(str)

def text_to_indices(text, char2idx):
    return [char2idx[c] for c in text]

def build_vocab(transcripts):
    texts = transcripts.astype(str)
    chars = sorted(set(''.join(texts)))
    char2idx = {c: i+1 for i, c in enumerate(chars)}  # 0 — CTC blank
    idx2char = {i: c for c, i in char2idx.items()}
    return char2idx, idx2char

char2idx, idx2char = build_vocab(df_train['transcription'])
vocab_size = len(char2idx) + 1  # include CTC blank

resampler = torchaudio.transforms.Resample(orig_freq=24000, new_freq=16000)
mel_spec  = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128)

class ASRDataset(Dataset):
    def __init__(self, df, data_base, char2idx):
        self.df = df
        self.base = data_base
        self.char2idx = char2idx
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        wav, sr = torchaudio.load(os.path.join(self.base, row['filename']))
        if sr != 16000:
            wav = resampler(wav)
        spec = mel_spec(wav).squeeze(0).transpose(0,1)  # T x n_mels
        target = torch.tensor(text_to_indices(row['transcription'], self.char2idx), dtype=torch.long)
        return spec, target

def collate_fn(batch):
    specs, targets = zip(*batch)
    spec_lens = [s.size(0) for s in specs]
    max_spec  = max(spec_lens)
    padded_specs = torch.zeros(len(specs), max_spec, specs[0].size(1))
    for i, s in enumerate(specs):
        padded_specs[i, :s.size(0)] = s
    tgt_lens = [t.size(0) for t in targets]
    targets_cat = torch.cat(targets)
    return padded_specs.transpose(1,2), torch.tensor(spec_lens), targets_cat, torch.tensor(tgt_lens)

train_ds = ASRDataset(df_train, DATA_DIR, char2idx)
dev_ds   = ASRDataset(df_dev,   DATA_DIR, char2idx)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, collate_fn=collate_fn)
dev_loader   = DataLoader(dev_ds,   batch_size=16, shuffle=False, collate_fn=collate_fn)

class ASRModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d((2,1)),
            nn.Conv2d(32,64,3, padding=1), nn.ReLU(), nn.MaxPool2d((2,1)),
            nn.Conv2d(64,64,3, padding=1), nn.ReLU(), nn.MaxPool2d((2,1)),
        )
        self.rnn = nn.LSTM(
            input_size=64*16,
            hidden_size=192,
            num_layers=2,
            batch_first=True,
            bidirectional=True,
        )
        self.classifier = nn.Linear(192*2, vocab_size)

    def forward(self, x):
        x = x.unsqueeze(1)  # (B,1,n_mels,T)
        x = self.cnn(x)     # (B,64,16,T′)
        B, C, H, W = x.size()
        x = x.permute(0,3,1,2).reshape(B, W, C*H)  # (B,T′,1024)
        x, _ = self.rnn(x)                       # (B,T′,384)
        x = self.classifier(x)                   # (B,T′,vocab)
        return x.log_softmax(2)

model = ASRModel(vocab_size)
num_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {num_params/1e6:.2f}M")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
ctc_loss  = nn.CTCLoss(blank=0, zero_infinity=True)

/usr/local/lib/python3.11/dist-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


Total parameters: 2.82M


In [3]:
import random
import copy
import torchaudio
# import torchaudio.sox_effects as sox_fx
import torchaudio.functional as F
from torchaudio.transforms import TimeMasking, FrequencyMasking
from tqdm import tqdm


time_mask = TimeMasking(time_mask_param=50)
freq_mask = FrequencyMasking(freq_mask_param=20)

def speed_perturb(wav, sr):
    # factor = random.choice([0.9, 1.0, 1.1])
    # effects = [['speed', str(factor)], ['rate', str(sr)]]
    # wav, _ = sox_fx.apply_effects_tensor(wav, sr, effects)
    # return wav
    factor = random.choice([0.9, 1.0, 1.1])
    new_sr = int(sr * factor)
    # down/up-sample to simulate speed change
    wav = F.resample(wav, orig_freq=sr, new_freq=new_sr)
    return F.resample(wav, orig_freq=new_sr, new_freq=sr)

# Augmented Dataset
class ASRAugDataset(ASRDataset):
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        wav, sr = torchaudio.load(os.path.join(self.base, row['filename']))
        if sr != 16000:
            wav = resampler(wav)
        # speed perturb
        if random.random() < 0.5:
            wav = speed_perturb(wav, 16000)
        # gaussian noise
        if random.random() < 0.5:
            wav = wav + 0.005 * torch.randn_like(wav)
        # random gain
        if random.random() < 0.5:
            gain = random.uniform(-6, +6)
            wav = torchaudio.functional.gain(wav, gain)
        # mel-spectrogram
        spec = mel_spec(wav).squeeze(0).transpose(0,1)
        # SpecAugment
        spec = time_mask(freq_mask(spec))
        target = torch.tensor(text_to_indices(row['transcription'], self.char2idx), dtype=torch.long)
        return spec, target

# recreate loader
train_aug_ds = ASRAugDataset(df_train, DATA_DIR, char2idx)
train_loader = DataLoader(train_aug_ds, batch_size=16, shuffle=True, collate_fn=collate_fn)

# scheduler and best-tracking
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
best_cer = float('inf')
best_state = None

def greedy_decode(log_probs):
    inds = log_probs.argmax(-1).cpu().numpy()
    seq, prev = [], 0
    for idx in inds:
        if idx and idx != prev:
            seq.append(idx2char[idx])
        prev = idx
    return ''.join(seq)

# run training+eval
num_epochs = 40
for epoch in range(1, num_epochs+1):
    model.train()
    total_loss = 0
    pbar = tqdm(train_loader, desc=f"Train E{epoch}/{num_epochs}", unit='batch')
    for specs, spec_lens, targets, tgt_lens in pbar:
        specs, targets = specs.to(device), targets.to(device)
        spec_lens, tgt_lens = spec_lens.to(device), tgt_lens.to(device)

        optimizer.zero_grad()
        lp = model(specs).permute(1,0,2)
        loss = ctc_loss(lp, targets, spec_lens, tgt_lens)
        loss.backward()
        optimizer.step()

        batch_loss = loss.item()
        total_loss += batch_loss
        lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(loss=f"{batch_loss:.4f}", lr=f"{lr:.2e}")
    avg_train = total_loss / len(train_loader)
    scheduler.step(avg_train)

    # eval
    model.eval()
    all_refs, all_hyps = [], []
    with torch.no_grad():
        for specs, spec_lens, targets, tgt_lens in tqdm(dev_loader, desc=" Eval", leave=False):
            specs = specs.to(device)
            lp = model(specs)
            flat = targets.tolist()
            lens = tgt_lens.tolist()
            off = 0
            for i, L in enumerate(lens):
                hy = greedy_decode(lp[i])
                ref_idxs = flat[off:off+L]; off += L
                rf = ''.join(idx2char[x] for x in ref_idxs)
                all_hyps.append(hy); all_refs.append(rf)
    cer_score = cer(all_refs, all_hyps)

    print(f"-> Epoch {epoch}: train loss {avg_train:.4f}, dev CER {cer_score:.4f}")
    if cer_score < best_cer:
        best_cer, best_state = cer_score, copy.deepcopy(model.state_dict())
        print(f"   *** New best Dev CER {best_cer:.4f} ***")

# load best model
model.load_state_dict(best_state)
print(f"\nLoaded best model with Dev CER={best_cer:.4f}")

Train E1/40: 100%|██████████| 785/785 [07:12<00:00,  1.82batch/s, loss=2.4713, lr=1.00e-03]


-> Epoch 1: train loss 2.8925, dev CER 1.0000
   *** New best Dev CER 1.0000 ***


Train E2/40: 100%|██████████| 785/785 [04:34<00:00,  2.86batch/s, loss=1.2983, lr=1.00e-03]


-> Epoch 2: train loss 2.0244, dev CER 0.7803
   *** New best Dev CER 0.7803 ***


Train E3/40: 100%|██████████| 785/785 [04:27<00:00,  2.94batch/s, loss=0.3520, lr=1.00e-03]


...

-> Epoch 38: train loss 0.0287, dev CER 0.1838


Train E39/40: 100%|██████████| 785/785 [04:32<00:00,  2.89batch/s, loss=0.0063, lr=5.00e-04]


-> Epoch 39: train loss 0.0254, dev CER 0.1827


Train E40/40: 100%|██████████| 785/785 [04:31<00:00,  2.89batch/s, loss=0.0043, lr=5.00e-04]
                                                        

-> Epoch 40: train loss 0.0274, dev CER 0.1909

Loaded best model with Dev CER=0.1777


In [4]:
torch.save(model.state_dict(), 'model_best.pth')

In [5]:
torch.save(best_state, 'best_model.pth')

model.load_state_dict(torch.load('best_model.pth', map_location=device))
model.to(device)

/tmp/ipykernel_19/756633090.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth', map_location=device))


ASRModel(
  (cnn): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (rnn): LSTM(1024, 192, num_layers=2, batch_first=True, bidirectional=True)
  (classifier): Linear(in_features=384, out_features=11, bias=True)
)

In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)  # мелкий LR
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

num_more_epochs = 5
for epoch in range(1, num_more_epochs+1):
    model.train()
    total_loss = 0
    for specs, spec_lens, targets, tgt_lens in tqdm(train_loader, desc=f"Fine-tune E{epoch}/{num_more_epochs}"):
        specs, targets = specs.to(device), targets.to(device)
        spec_lens, tgt_lens = spec_lens.to(device), tgt_lens.to(device)

        optimizer.zero_grad()
        lp = model(specs).permute(1,0,2)
        loss = ctc_loss(lp, targets, spec_lens, tgt_lens)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg = total_loss / len(train_loader)
    scheduler.step(avg)

    model.eval()
    all_refs, all_hyps = [], []
    with torch.no_grad():
        for specs, spec_lens, targets, tgt_lens in dev_loader:
            specs = specs.to(device)
            lp = model(specs)
            off = 0
            flat = targets.tolist()
            for i,L in enumerate(tgt_lens.tolist()):
                hy = greedy_decode(lp[i])
                rf = ''.join(idx2char[x] for x in flat[off:off+L])
                off += L
                all_hyps.append(hy); all_refs.append(rf)
    cer_score = cer(all_refs, all_hyps)
    print(f"Fine-tune Epoch {epoch}: train loss {avg:.4f}, dev CER {cer_score:.4f}")

Fine-tune E1/5: 100%|██████████| 785/785 [04:31<00:00,  2.89it/s]


Fine-tune Epoch 1: train loss 0.0393, dev CER 0.1775


Fine-tune E2/5: 100%|██████████| 785/785 [04:30<00:00,  2.91it/s]


Fine-tune Epoch 2: train loss 0.0394, dev CER 0.1772


Fine-tune E3/5: 100%|██████████| 785/785 [04:30<00:00,  2.90it/s]


Fine-tune Epoch 3: train loss 0.0348, dev CER 0.1765


Fine-tune E4/5: 100%|██████████| 785/785 [04:27<00:00,  2.94it/s]


Fine-tune Epoch 4: train loss 0.0351, dev CER 0.1762


Fine-tune E5/5: 100%|██████████| 785/785 [04:28<00:00,  2.93it/s]


Fine-tune Epoch 5: train loss 0.0324, dev CER 0.1770


In [7]:
import os
import pandas as pd
import torch
import torchaudio
from tqdm import tqdm


BASE     = '/kaggle/input/asr-numbers-recognition-in-russian'
TEST_CSV = os.path.join(BASE, 'test.csv')
OUT_CSV  = 'predictions.csv'

df = pd.read_csv(TEST_CSV) 

model.eval()
preds = []
for fn in tqdm(df['filename'], desc='Inference'):
    wav, sr = torchaudio.load(os.path.join(BASE, fn))
    if sr != 16000:
        wav = resampler(wav)
    spec = mel_spec(wav).squeeze(0).unsqueeze(0).to(device)
    with torch.no_grad():
        logp = model(spec)
    preds.append(greedy_decode(logp[0]))

out = pd.DataFrame({
    'filename': df['filename'],
    'transcription': preds
})

out['transcription'] = out['transcription'].fillna(0)

n_null = out['transcription'].isna().sum()
print(f"Nulls in transcription: {n_null}")

out['transcription'] = pd.to_numeric(out['transcription'], errors='coerce').fillna(0).astype(int)
out['transcription'] = out['transcription'].apply(lambda x: int(x) if x >= 0 else 0)

out.to_csv(OUT_CSV, index=False)
print(f"Saved → {OUT_CSV}")

Inference: 100%|██████████| 2582/2582 [01:47<00:00, 24.07it/s]


Nulls in transcription: 0
Saved → predictions.csv


In [8]:
torch.save(model.state_dict(), 'model_best_2.pth')